#Imports

In [ ]:
import pandas as pd
import numpy as np
import collections


import warnings
warnings.filterwarnings('ignore')
import glob
import numpy as np
import pandas as pd
import itertools


from datetime import datetime
from sklearn import preprocessing
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

from numpy import random

import joblib

#Preprocessing the data


P VALUE PREPROCESSING

In [ ]:
#The data should have snp number as column header, along with the phenotype(disease or no disease data), along with the p value and the first and second allele

In [ ]:
file = '/content/drive/MyDrive/Identifying Diseases Using ML /preprocessed_matrix.csv'
data = pd.read_csv(file, sep = ' ')

data = data.set_index('usersid')
snps = data.columns
snps = snps[:-1]
data = data[snps]
n_samples = 128

In [ ]:
data.head()

,rs7541950,rs7541956,rs7542025,rs7542028,rs7542082,rs754217,rs7542208,rs7542239,rs754224,rs7542242,...,rs751062,rs751082,rs751104,rs751114,rs7511209,rs751128,rs7511306,rs7511364,rs751137,rs7511662
usersid,,,,,,,,,,,,,,,,,,,,,
user123,TT,GG,AG,CT,TT,CT,AG,AG,GG,CT,...,CT,CC,GT,GG,GG,TT,AA,AA,AA,CC
user285,TT,GG,AG,TT,TT,CC,AG,AG,GT,CT,...,CT,AC,GG,AG,GG,CT,AG,AG,AC,CC
user2849,CT,AG,AG,CC,CT,CC,AG,AA,GG,CC,...,CT,AC,TT,AG,GG,CT,GG,AG,AA,TT
user1,CC,GG,GG,CC,TT,CT,AG,AG,GG,CT,...,CC,AC,GT,AG,GG,CT,AA,GG,CC,CC
user26,CT,GG,GG,CC,CT,CT,GG,GG,GT,TT,...,CC,CC,TT,AG,GG,CT,GG,AA,AA,CT


In [ ]:
allele_freq = []
hetero = 0
ind_hetero = []
counter = 0

for snp in snps:
    print('SNP NUMBER:', counter)
    genotype_count = data[snp].value_counts().to_dict()     #Take the snp columns, and get the unique value counts ie the frequency of the users
    genotypes = data[snp].unique()                          #Genotypes is the array of unique snps               
    n_genotypes = len(genotypes)                            #Count of different types of genotypes
    genotypes = sorted(genotypes)
    if n_genotypes > 3:                                     #This condition is for making the heterozygous genotypes consistent
        het1 = genotype_count[genotypes[1]]                 #Count of hetero 1 
        het2 = genotype_count[genotypes[2]]                 #Count of hetero 2
        hetero = het1 + het2                                #Count of both heteros together
        genotype_count[genotypes[1]] = hetero               
        del genotype_count[genotypes[2]]
        genotypes.remove(genotypes[2])
    
    for key in genotype_count:                          #This is for normalizing the count?
        genotype_count[key] = genotype_count[key]/n_samples
    
    print('Observation', genotype_count)
    genotype_count = collections.OrderedDict(sorted(genotype_count.items()))

    if len(genotypes) == 3:                             #All three are present, by frequency count which one is more, set that as the primary allele with the class number 1
        allele_A = genotype_count[genotypes[0]] + (genotype_count[genotypes[1]] * 0.5)
        allele_B = genotype_count[genotypes[2]] + (genotype_count[genotypes[1]] * 0.5)
        print('allele:', allele_A)
        print('allele', allele_B)
        if allele_B > allele_A:
            encode = np.array([3,2,1])
        else:
            encode = np.array([1,2,3])
    
    elif len(genotypes) == 2:               #If only two homozygous or one homozygous alleles are present
        encode = np.array([1,2])
    else:
        encode = np.array([1,2])
    
    genotype_dict = dict(zip(genotypes, encode))    
    data[snp] = data[snp].map(genotype_dict)
    print(data[snp].head())
    counter = counter + 1
    # if counter>5000:
    #     break

Streaming output truncated to the last 5000 lines.
user26      1
Name: rs17017593, dtype: int64
SNP NUMBER: 65928
Observation {'GG': 0.8046875, 'GT': 0.1640625, 'TT': 0.03125}
allele: 0.88671875
allele 0.11328125
usersid
user123     1
user285     1
user2849    1
user1       1
user26      1
Name: rs17017634, dtype: int64
SNP NUMBER: 65929
Observation {'GG': 0.8046875, 'AG': 0.15625, 'AA': 0.0390625}
allele: 0.1171875
allele 0.8828125
usersid
user123     1
user285     1
user2849    1
user1       1
user26      1
Name: rs17017668, dtype: int64
SNP NUMBER: 65930
Observation {'TT': 0.609375, 'CT': 0.3359375, 'CC': 0.0546875}
allele: 0.22265625
allele 0.77734375
usersid
user123     1
user285     1
user2849    1
user1       2
user26      2
Name: rs17017733, dtype: int64
SNP NUMBER: 65931
Observation {'TT': 0.6953125, 'CT': 0.2421875, 'CC': 0.0625}
allele: 0.18359375
allele 0.81640625
usersid
user123     1
user285     1
user2849    1
user1       2
user26      2
Name: rs17017756, dtype: int64
SN

In [ ]:
data["Class"] = 0
for i in range(len(data)):
    data["Class"][i] = random.randint(2)

#print(data["Class"].nunique())

In [ ]:
outfile = '/content/drive/MyDrive/Identifying Diseases Using ML /encoded_allsnp.csv'
data.to_csv(outfile,sep=' ')

In [ ]:
outfile = '/content/drive/MyDrive/Identifying Diseases Using ML /encoded_allsnp2.csv'
data.to_csv(outfile,sep=',')

In [ ]:
outfile = 'encoded_allsnp.csv'
data.to_csv(outfile, sep = ' ')

In [ ]:
outfile = 'encoded_allsnp2.csv'
data.to_csv(outfile, sep = ',')

#Functions


In [ ]:
def convertLabel(data):
	label_maker = LabelEncoder()
	data['Encode'] = label_maker.fit_transform(data['Class'])

In [ ]:
def knn(X_train, y_train, X_test):
	scaler = StandardScaler()  
	scaler.fit(X_train)

	X_train = scaler.transform(X_train)  
	X_test = scaler.transform(X_test)

	classifier = KNeighborsClassifier(n_neighbors=k)  
	classifier.fit(X_train, y_train) 
	return classifier.predict(X_test)

In [ ]:
def GBC(X_train, y_train, X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    classifier = GradientBoostingClassifier(n_estimators=500,learning_rate=0.05,random_state=100,max_features=5)
    classifier.fit(X_train,y_train)
    return classifier.predict(X_test)

#RF-Model

##RF KNN

In [ ]:
#first use rf for snp feature selection
#Then apply KNN

In [ ]:
start = datetime.now()
data_file = '/content/drive/MyDrive/Identifying Diseases Using ML /encoded_classallsnp.csv'         #Get the encoded and preprocessed snp dataset
data = pd.read_csv(data_file, sep = ' ')                                                        
data = data.drop('usersid', axis=1)
data = data.replace(np.NaN, 0)                      #Remove illegal values

snp = data.columns.values
snp = snp[:-1]
X = data[snp].values

In [ ]:
correct = 0
mismatch = 0
y_pred = []
n_features = 400
k = 7                       #Number of clusters
counter = 0
n_samples = 128
y_true = []

In [ ]:
convertLabel(data)
y = data['Encode'].values

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)

for train_index, test_index in loo.split(X):						#Use cross validation and make multiple splits of the datasets
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]

	fs = RandomForestClassifier(max_features = 500, n_estimators=200, random_state=42)				#Make a random forest classifier
	fs.fit(X_train, y_train)													
	ranking = dict(zip(snp, fs.feature_importances_))												#Take the top snps
	ranking = dict(itertools.islice(ranking.items(), n_features))											
	top_snp = list(ranking.keys())																#Take the top snps
	top_snp = np.asarray(top_snp)

	X_train = data.drop(test_index)																#Create training and testing data 
	X_train = X_train[top_snp].values
	X_test = data.reindex(test_index)
	X_test = X_test[top_snp].values

	tmp = knn(X_train, y_train, X_test)														#Use knn classifier
	
	y_pred.append(tmp[0])
	y_true.append(y_test)
	print (counter, '===========================')
	print('Prediction:', tmp)
	print('True label', y_test[0])

	counter = counter + 1
	

y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print('Number of SNPs:', n_features)
print('Accuracy using of rf-knn', accuracy)
print('Precision using of rf-knn', accuracy)
print('Recall using of rf-knn', accuracy)
print('\nConfusion matrix of k-nearest neighbors (kNN) optimized on the test data:')
print(pd.DataFrame(confusion_matrix(y_true, y_pred),
      columns=['pred_pos', 'pred_neg'], index=['pos', 'neg']))
print('Run-time', datetime.now() - start)

0 ===========================
Prediction: [0]
True label 0
1 ===========================
Prediction: [0]
True label 0
2 ===========================
Prediction: [0]
True label 0
3 ===========================
Prediction: [0]
True label 0
4 ===========================
Prediction: [0]
True label 0
5 ===========================
Prediction: [0]
True label 0
6 ===========================
Prediction: [0]
True label 0
7 ===========================
Prediction: [0]
True label 0
8 ===========================
Prediction: [0]
True label 0
9 ===========================
Prediction: [0]
True label 0
10 ===========================
Prediction: [0]
True label 0
11 ===========================
Prediction: [0]
True label 0
12 ===========================
Prediction: [0]
True label 0
13 ===========================
Prediction: [0]
True label 0
14 ===========================
Prediction: [0]
True label 0
15 ===========================
Prediction: [0]
True label 0
16 ===========================
Prediction: [0]
Tru

##RF SVM

In [ ]:
start = datetime.now()
data_file = '/content/drive/MyDrive/Identifying Diseases Using ML /encoded_classallsnp.csv'         #Get the encoded and preprocessed snp dataset
data = pd.read_csv(data_file, sep = ' ')                                                        
data = data.drop('usersid', axis=1)
data = data.replace(np.NaN, 0)                      #Remove illegal values

snp = data.columns.values
snp = snp[:-1]
X = data[snp].values

In [ ]:
kernel = 'rbf'
c = 100
ep = 0.001

In [ ]:
correct = 0
mismatch = 0
y_pred = []
n_features = 310
counter = 0
n_samples = 128
y_true = []

In [ ]:
convertLabel(data)
y = data['Encode'].values

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)

for train_index, test_index in loo.split(X):                            #Use cross validation and make multiple splits of the datasets
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]

	fs = RandomForestClassifier(max_depth = 3, n_estimators=200, random_state=42)       #Make a random forest classifier
	fs.fit(X_train, y_train)
	ranking = dict(zip(snp, fs.feature_importances_))

	ranking = dict(itertools.islice(ranking.items(), n_features))                   #Take the top snps
	top_snp = list(ranking.keys())
	top_snp = np.asarray(top_snp)
	selected_snp = pd.DataFrame(top_snp)
	outfile = '/content/drive/MyDrive/Identifying Diseases Using ML /rf-svm/snp' + str(counter) + '.csv'
	selected_snp.to_csv(outfile, sep=' ', index=False)

	X_train = data.drop(test_index)                                         #Create training and testing data
	X_train = X_train[top_snp].values
	X_test = data.reindex(test_index)
	X_test = X_test[top_snp].values

	clf = SVC()                                                 #SVM 
	#clf = SVC(kernel=kernel, gamma=ep, C=c)
	clf.fit(X_train, y_train)
	tmp = clf.predict(X_test)
	y_pred.append(tmp)
	y_true.append(y_test)
	print (counter, '===========================')
	print('Prediction:', tmp)
	print('True label', y_test[0])

	counter = counter + 1

y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
#print('ORDER:', order)
print('Number of SNP', n_features)
print('Parameter kernel', kernel)
print('Parameter gamma', ep)
print('Parameter C', c)
print('Accuracy using rf-svm', accuracy)
print('Precision using rf-svm', precision)
print('Recall using rf-svm', recall)
print('\nConfusion matrix of support vector machine (SVM) optimized on the test data:')
print(pd.DataFrame(confusion_matrix(y_true, y_pred),
    columns=['pred_pos', 'pred_neg'], index=['pos', 'neg']))
print('Run-time', datetime.now() - start)

0 ===========================
Prediction: [0]
True label 0
1 ===========================
Prediction: [0]
True label 0
2 ===========================
Prediction: [0]
True label 0
3 ===========================
Prediction: [0]
True label 0
4 ===========================
Prediction: [0]
True label 0
5 ===========================
Prediction: [0]
True label 0
6 ===========================
Prediction: [0]
True label 0
7 ===========================
Prediction: [0]
True label 0
8 ===========================
Prediction: [0]
True label 0
9 ===========================
Prediction: [0]
True label 0
10 ===========================
Prediction: [0]
True label 0
11 ===========================
Prediction: [0]
True label 0
12 ===========================
Prediction: [0]
True label 0
13 ===========================
Prediction: [0]
True label 0
14 ===========================
Prediction: [0]
True label 0
15 ===========================
Prediction: [0]
True label 0
16 ===========================
Prediction: [0]
Tru

##RF-GBT

In [ ]:
start = datetime.now()
data_file = '/content/drive/MyDrive/Identifying Diseases Using ML /encoded_classallsnp.csv'         #Get the encoded and preprocessed snp dataset
data = pd.read_csv(data_file, sep = ' ')                                                        
data = data.drop('usersid', axis=1)
data = data.replace(np.NaN, 0)                      #Remove illegal values

snp = data.columns.values
snp = snp[:-1]
X = data[snp].values

In [ ]:
correct = 0
mismatch = 0
y_pred = []
n_features = 400
k = 7                       #Number of clusters
counter = 0
n_samples = 128
y_true = []

In [ ]:
# classifier = GradientBoostingClassifier(criterion='friedman_mse',init=None,learning_rate=0.05,
    #                                         loss='deviance',max_depth=3,max_features=5,min_samples_split=2,n_estimaors=500)
    #                                         max_leaf_nodes=None,
    #                                         min_impurity_decrease=0.0,min_samples_leaf=1,
    #                                         min_samples_split=2,min_weight_fraction_leaf=0.0,n_estimators=500,
    #                                         n_iter_no_change=None,presort='auto',random_state=100,subsample=1.0,tol=0.0001,validation_fraction=0.1,verbose=0,warm_start=False)
    # classifier = GradientBoostingClassifier(criterion='friedman_mse',init=None,learning_rate=0.05, 
    #                                         loss='deviance', max_depth=3,max_features=5, max_leaf_nodes=None,
    #                                         min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1,
    #                                         min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=500,n_iter_no_change=None, 
    #                                         presort='auto',random_state=100, subsample=1.0, tol=0.0001,validation_fraction=0.1, 
    #                                         verbose=0,warm_start=False)

In [ ]:
convertLabel(data)
y = data['Encode'].values

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)

for train_index, test_index in loo.split(X):						#Use cross validation and make multiple splits of the datasets
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]

	fs = RandomForestClassifier(max_features = 500, n_estimators=200, random_state=42)				#Make a random forest classifier
	fs.fit(X_train, y_train)													
	ranking = dict(zip(snp, fs.feature_importances_))												#Take the top snps
	ranking = dict(itertools.islice(ranking.items(), n_features))											
	top_snp = list(ranking.keys())																#Take the top snps
	top_snp = np.asarray(top_snp)

	X_train = data.drop(test_index)																#Create training and testing data 
	X_train = X_train[top_snp].values
	X_test = data.reindex(test_index)
	X_test = X_test[top_snp].values

	tmp = GBC(X_train, y_train, X_test)														#Use knn classifier
	
	y_pred.append(tmp[0])
	y_true.append(y_test)
	print (counter, '===========================')
	print('Prediction:', tmp)
	print('True label', y_test[0])

	counter = counter + 1
	

y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print('Number of SNPs:', n_features)
print('Accuracy using of rf-knn', accuracy)
print('Precision using of rf-knn', accuracy)
print('Recall using of rf-knn', accuracy)
print('\nConfusion matrix of k-nearest neighbors (kNN) optimized on the test data:')
print(pd.DataFrame(confusion_matrix(y_true, y_pred),
      columns=['pred_pos', 'pred_neg'], index=['pos', 'neg']))
print('Run-time', datetime.now() - start)

0 ===========================
Prediction: [0]
True label 0
1 ===========================
Prediction: [0]
True label 0
2 ===========================
Prediction: [0]
True label 0
3 ===========================
Prediction: [0]
True label 0
4 ===========================
Prediction: [0]
True label 0
5 ===========================
Prediction: [0]
True label 0
6 ===========================
Prediction: [0]
True label 0
7 ===========================
Prediction: [0]
True label 0
8 ===========================
Prediction: [0]
True label 0
9 ===========================
Prediction: [0]
True label 0
10 ===========================
Prediction: [0]
True label 0
11 ===========================
Prediction: [0]
True label 0
12 ===========================
Prediction: [0]
True label 0
13 ===========================
Prediction: [0]
True label 0
14 ===========================
Prediction: [0]
True label 0
15 ===========================
Prediction: [0]
True label 0
16 ===========================
Prediction: [0]
Tru

#Lasso Model

##Lasso KNN

In [ ]:
start = datetime.now()
data_file = '/content/drive/MyDrive/Identifying Diseases Using ML /encoded_classallsnp.csv'         #Get the encoded and preprocessed snp dataset
data = pd.read_csv(data_file, sep = ' ')                                                        
data = data.drop('usersid', axis=1)
data = data.replace(np.NaN, 0)                      #Remove illegal values

snp = data.columns.values
snp = snp[:-1]
X = data[snp].values

In [ ]:
pipeline = Pipeline([
                     ('scaler',StandardScaler),
                     ('model',Lasso())
])

In [ ]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,2,0.1)},
                      cv=5,
                      scoring = 'neg_mean_squared_error',
                      verbose=3
                      )

In [ ]:
convertLabel(data)
y = data['Encode'].values

NameError: ignored

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_

In [ ]:
coef = search.best_estimator_[1].coef_
importance = np.abs(coef)

In [ ]:
print(importance)

In [ ]:
np.array(features)[coef!=0]         #  selected features

In [ ]:
np.array(features)[coef==0]         #  discarded features

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)

for train_index, test_index in loo.split(X):						#Use cross validation and make multiple splits of the datasets
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]

	fs = RandomForestClassifier(max_features = 500, n_estimators=200, random_state=42)				#Make a random forest classifier
	search.fit(X_train, y_train)													
	ranking = dict(zip(snp, fs.feature_importances_))												#Take the top snps
	ranking = dict(itertools.islice(ranking.items(), n_features))											
	top_snp = list(ranking.keys())																#Take the top snps
	top_snp = np.asarray(top_snp)

	X_train = data.drop(test_index)																#Create training and testing data 
	X_train = X_train[top_snp].values
	X_test = data.reindex(test_index)
	X_test = X_test[top_snp].values

	tmp = knn(X_train, y_train, X_test)														#Use knn classifier
	
	y_pred.append(tmp[0])
	y_true.append(y_test)
	print (counter, '===========================')
	print('Prediction:', tmp)
	print('True label', y_test[0])

	counter = counter + 1
	

y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print('Number of SNPs:', n_features)
print('Accuracy using of rf-knn', accuracy)
print('Precision using of rf-knn', accuracy)
print('Recall using of rf-knn', accuracy)
print('\nConfusion matrix of k-nearest neighbors (kNN) optimized on the test data:')
print(pd.DataFrame(confusion_matrix(y_true, y_pred),
      columns=['pred_pos', 'pred_neg'], index=['pos', 'neg']))
print('Run-time', datetime.now() - start)

##Lasso SVM

In [ ]:
pipeline = Pipeline([
                     ('scaler',StandardScaler),
                     ('model',Lasso())
])

In [ ]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,2,0.1)},
                      cv=5,
                      scoring = 'neg_mean_squared_error',
                      verbose=3
                      )

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_

In [ ]:
coef = search.best_estimator_[1].coef_

In [ ]:
np.array(features)[coef!=0]         #  selected features

In [ ]:
np.array(features)[coef==0]         #  discarded features

In [ ]:
# def convertLabel(data):
# 	label_maker = LabelEncoder()
# 	data['Encode'] = label_maker.fit_transform(data['Class'])

loo = LeaveOneOut()
loo.get_n_splits(X)

for train_index, test_index in loo.split(X):                            #Use cross validation and make multiple splits of the datasets
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]

	fs = RandomForestClassifier(max_depth = 3, n_estimators=200, random_state=42)       #Make a random forest classifier
	fs.fit(X_train, y_train)
	ranking = dict(zip(snp, fs.feature_importances_))

	ranking = dict(itertools.islice(ranking.items(), n_features))                   #Take the top snps
	top_snp = list(ranking.keys())
	top_snp = np.asarray(top_snp)
	selected_snp = pd.DataFrame(top_snp)
	outfile = 'rf-svm/snp' + str(counter) + '.csv'
	selected_snp.to_csv(outfile, sep=' ', index=False)

	X_train = data.drop(test_index)                                         #Create training and testing data
	X_train = X_train[top_snp].values
	X_test = data.reindex(test_index)
	X_test = X_test[top_snp].values

	clf = SVC()                                                 #SVM 
	#clf = SVC(kernel=kernel, gamma=ep, C=c)
	clf.fit(X_train, y_train)
	tmp = clf.predict(X_test)
	y_pred.append(tmp)
	y_true.append(y_test)
	print (counter, '===========================')
	print('Prediction:', tmp)
	print('True label', y_test[0])

	counter = counter + 1

y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print('ORDER:', order)
print('Number of SNP', n_features)
print('Parameter kernel', kernel)
print('Parameter gamma', ep)
print('Parameter C', c)
print('Accuracy using rf-svm', accuracy)
print('Precision using rf-svm', precision)
print('Recall using rf-svm', recall)
print('\nConfusion matrix of support vector machine (SVM) optimized on the test data:')
print(pd.DataFrame(confusion_matrix(y_true, y_pred),
    columns=['pred_pos', 'pred_neg'], index=['pos', 'neg']))
print('Run-time', datetime.now() - start)

##Lasso GBT

In [ ]:
pipeline = Pipeline([
                     ('scaler',StandardScaler),
                     ('model',Lasso())
])

In [ ]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,2,0.1)},
                      cv=5,
                      scoring = 'neg_mean_squared_error',
                      verbose=3
                      )

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_

In [ ]:
coef = search.best_estimator_[1].coef_

In [ ]:
np.array(features)[coef!=0]         #  selected features

In [ ]:
np.array(features)[coef==0]         #  discarded features

#EXTRAS

##S1

In [ ]:
import pandas as pd
import numpy as np
import collections



In [ ]:
"""" 
	This script represents SNP as 1, 2, 3 for homozygous, heterozygous, and
	variant homozygous, respectively.
	Input:
		file: dataset/preprocessed_matrix.csv
				Cleaned SNP dataset
	
	Output:
		file: dataset/encoded_snp.csv
				Encoded SNP dataset
""""

file = 'dataset/preprocessed_matrix.csv'
data = pd.read_csv(file, sep = ' ')
data = data.set_index('usersid')
snps = data.columns
snps = snps[:-1]
data = data[snps]
n_samples = 128

allele_freq = []
hetero = 0
ind_hetero = []
counter = 0

for snp in snps:
	print('SNP NUMBER:', counter)
	genotype_count = data[snp].value_counts().to_dict()
	genotypes = data[snp].unique()
	n_genotypes = len(genotypes)
	genotypes = sorted(genotypes)

	if n_genotypes > 3:
		het1 = genotype_count[genotypes[1]]
		het2 = genotype_count[genotypes[2]]
		hetero = het1 + het2
		genotype_count[genotypes[1]] = hetero
		del genotype_count[genotypes[2]]
		genotypes.remove(genotypes[2])

	for key in genotype_count:
		genotype_count[key] = genotype_count[key]/n_samples
	print('Observation', genotype_count)
	genotype_count = collections.OrderedDict(sorted(genotype_count.items()))

	if len(genotypes) == 3:
		allele_A = genotype_count[genotypes[0]] + (genotype_count[genotypes[1]] * 0.5)
		allele_B = genotype_count[genotypes[2]] + (genotype_count[genotypes[1]] * 0.5)
		print('allele:', allele_A)
		print('allele', allele_B)
		if allele_B > allele_A:
			encode = np.array([3,2,1])
		else:
			encode = np.array([1,2,3])
	elif len(genotypes) == 2:
		encode = np.array([1,2])
	else:
		encode = np.array([1])

	genotype_dict = dict(zip(genotypes, encode))
	data[snp] = data[snp].map(genotype_dict)

	print(data[snp].head())
	counter = counter + 1

outfile = 'encoded_snp.csv'
data.to_csv(outfile, sep = ' ')

##S2

In [ ]:
DATAmap=pd.read_table('DATAjune.map',header=None)
ALL_SNPs_names=DATAmap[1].values
ALL_SNPs_chrs=DATAmap[0].values
ALL_SNPs_pos=DATAmap[3].values

In [ ]:
SNP_balls=np.load('SNP_all_balls.npy')
SNP_Jostins_balls=np.load('SNP_Jostins_balls.npy')
SNP_Jostins_balls_order=np.load('SNP_Jostins_balls_order.npy')

In [ ]:
All_SNP_QC=np.load('SNP_QC.npy')

presel_QC=np.unique((np.load('trainpresel_QC_union.npy')/2).astype(int))

SNP_QC=All_SNP_QC[presel_QC]

In [ ]:
# Number of fold for each model
Nfold=10

In [ ]:
weight_LR=np.load('feat_select_LR_QC_rand_HWcontrol_all_FOLD.npy')
gain_LGBM=np.load('feat_select_LGBM_QC_rand_HWcontrol_all_FOLD.npy')
gain_resnet=np.load('feat_select_ResNet_QC_rand_HWcontrol_all_FOLD.npy')
random_coeff=np.load('feat_select_rand_QC_rand_HWcontrol_all_FOLD.npy')
weight_Jostins_norm=np.load('weight_Jostins_norm.npy')

finding base on the positions

In [ ]:
# Defining globally defined windows of 500Kb starting from the beginning of DNA
size_wind=500000
# size_wind=200000
# size_wind=100000

In [ ]:
init_chrom=np.zeros((23,2))
init_fin=np.zeros(23)
for i in range(23):
    init_chrom[i,0]=np.sum(SNP_balls[:,0]<i+1).astype(int)
for i in range(1,23):  
    init_fin[i]=SNP_balls[(init_chrom[i,0]-1).astype(int),2]
init_fin=init_fin.astype(int)

init_chrom=init_chrom.astype(int)

correction_new_chrom=2*size_wind
for i in range(1,23):
    aaa=0
    for j in range(0,i+1):
        aaa=init_fin[j]+aaa+correction_new_chrom
    init_chrom[i,1]=aaa


In [ ]:
SNP_balls_centers=np.zeros((SNP_QC.shape[0],2))
SNP_balls_centers[:,0]=np.copy(SNP_balls[SNP_QC][:,0])
SNP_balls_centers[:,1]=np.copy((SNP_balls[SNP_QC][:,2]+SNP_balls[SNP_QC][:,1])/2)
SNP_balls_centers=SNP_balls_centers.astype(int)

In [ ]:
# coord of SNPs preselected, in terms of basis from beginning DNA
xxx=np.zeros(SNP_balls_centers.shape[0])
for i in range(SNP_balls_centers.shape[0]):
    xxx[i]=SNP_balls_centers[i,1]+init_chrom[SNP_balls_centers[i,0]-1,1]

In [ ]:
SNP_Jostins_balls_centers=np.zeros((SNP_Jostins_balls_order.shape[0],2))
SNP_Jostins_balls_centers[:,0]=np.copy(SNP_Jostins_balls_order[:,0])
SNP_Jostins_balls_centers[:,1]=np.copy((SNP_Jostins_balls_order[:,2]+SNP_Jostins_balls_order[:,1])/2)
SNP_Jostins_balls_centers=SNP_Jostins_balls_centers.astype(int)


In [ ]:
# coord of SNPs GWAS, in terms of basis from beginning DNA
xxx_jost=np.zeros(SNP_Jostins_balls_centers.shape[0])
for i in range(xxx_jost.shape[0]):
    xxx_jost[i]=SNP_Jostins_balls_centers[i,1]+init_chrom[SNP_Jostins_balls_centers[i,0]-1,1]

Building Windows

In [ ]:
LLL=xxx[-1]
x_basis_borders=np.arange(0,LLL+size_wind,size_wind)
x_basis=x_basis_borders[:-1]+size_wind/2

x_basis_borders_shift=np.arange(0,LLL+size_wind,size_wind)+size_wind/2
x_basis_shift=x_basis_borders_shift[:-1]+size_wind/2

Position base for presels and jostins

In [ ]:
# I build the mean on the folds of the coefficients 
y_LR_prel=np.mean(weight_LR,axis=0)
y_LGBM_prel=np.mean(gain_LGBM,axis=0)
y_ResNet_prel=np.mean(gain_resnet,axis=0)
y_Jost_prel=weight_Jostins_norm
y_rand_prel=np.mean(random_coeff,axis=0)

In [ ]:
# In which window, starting from the beginning, each SNP preselected is located
y_presel_pos=np.zeros(y_LR_prel.shape[0])

for j in range(y_LR_prel.shape[0]):
    y_presel_pos[j]=np.nonzero((xxx[j]>x_basis_borders))[0][-1]

y_presel_pos=y_presel_pos.astype(int)

# In which window, starting from the beginning, each SNP preselected is located - shifted by half window
y_presel_pos_shift=np.zeros(y_LR_prel.shape[0])

for j in range(y_LR_prel.shape[0]):
    y_presel_pos_shift[j]=np.nonzero((xxx[j]>x_basis_borders_shift))[0][-1]

y_presel_pos_shift=y_presel_pos_shift.astype(int)

In [ ]:
print('Number of independent windows for preselected SNPs: ',
      np.unique(y_presel_pos).shape[0])

In [ ]:
# In which window, starting from the beginning, each SNP of GWAS is located
y_Jost_pos=np.zeros(y_Jost_prel.shape[0])

for j in range(y_Jost_prel.shape[0]):
    y_Jost_pos[j]=np.nonzero((xxx_jost[j]>x_basis_borders))[0][-1]

y_Jost_pos=y_Jost_pos.astype(int)

# In which window, starting from the beginning, each SNP of Jostins is located - shifted
y_Jost_pos_shift=np.zeros(y_Jost_prel.shape[0])

for j in range(y_Jost_prel.shape[0]):
    y_Jost_pos_shift[j]=np.nonzero((xxx_jost[j]>x_basis_borders_shift))[0][-1]

y_Jost_pos_shift=y_Jost_pos_shift.astype(int)

In [ ]:
y_basis_count=np.zeros(x_basis.shape[0])
for i in range(x_basis.shape[0]):
    y_basis_count[i]=np.sum(y_presel_pos==i)
    
y_presel_count=y_basis_count[y_basis_count>0]

##S3


In [ ]:
import scipy.io as sio  

In [ ]:
genomat = "/content/genodata.mat"
data_1=sio.loadmat(genomat)

In [ ]:
gen = data_1.keys()

In [ ]:
data_1["genotest"]

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

##S4

In [ ]:
import pandas as pd
import numpy as np
import collections

# """
# 	This script represents SNP as 1, 2, 3 for homozygous, heterozygous, and
# 	variant homozygous, respectively.
# 	Input:
# 		file: dataset/preprocessed_matrix.csv
# 				Cleaned SNP dataset
	
# 	Output:
# 		file: dataset/encoded_snp.csv
# 				Encoded SNP dataset
# """

In [ ]:
file = '/content/drive/MyDrive/Identifying Diseases Using ML /preprocessed_matrix.csv'
data = pd.read_csv(file, sep = ' ')

data = data.set_index('usersid')
snps = data.columns
snps = snps[:-1]
data = data[snps]
n_samples = 128

In [ ]:
data.head()

,rs7541950,rs7541956,rs7542025,rs7542028,rs7542082,rs754217,rs7542208,rs7542239,rs754224,rs7542242,...,rs751062,rs751082,rs751104,rs751114,rs7511209,rs751128,rs7511306,rs7511364,rs751137,rs7511662
usersid,,,,,,,,,,,,,,,,,,,,,
user123,TT,GG,AG,CT,TT,CT,AG,AG,GG,CT,...,CT,CC,GT,GG,GG,TT,AA,AA,AA,CC
user285,TT,GG,AG,TT,TT,CC,AG,AG,GT,CT,...,CT,AC,GG,AG,GG,CT,AG,AG,AC,CC
user2849,CT,AG,AG,CC,CT,CC,AG,AA,GG,CC,...,CT,AC,TT,AG,GG,CT,GG,AG,AA,TT
user1,CC,GG,GG,CC,TT,CT,AG,AG,GG,CT,...,CC,AC,GT,AG,GG,CT,AA,GG,CC,CC
user26,CT,GG,GG,CC,CT,CT,GG,GG,GT,TT,...,CC,CC,TT,AG,GG,CT,GG,AA,AA,CT


In [ ]:
# snp = snps[0]
# y = data[snp].value_counts().to_dict()
# x = sorted(data[snp].unique())

In [ ]:
# h1 = y[x[1]]
# h2 = y[x[2]]
# h1+h2

# sorted(y.items())

# aA = y[x[0]]+ 0.5*y[x[1]]
# aB = y[x[2]]+ 0.5*y[x[1]]

# print('alleleA', aA)
# print('alleleB', aB)

In [ ]:
allele_freq = []
hetero = 0
ind_hetero = []
counter = 0

for snp in snps:
    print('SNP NUMBER:', counter)
    genotype_count = data[snp].value_counts().to_dict()     #Take the snp columns, and get the unique value counts ie the frequency of the users
    genotypes = data[snp].unique()                          #Genotypes is the array of unique snps               
    n_genotypes = len(genotypes)                            #Count of different types of genotypes
    genotypes = sorted(genotypes)
    if n_genotypes > 3:                                     #This condition is for making the heterozygous genotypes consistent
        het1 = genotype_count[genotypes[1]]                 #Count of hetero 1 
        het2 = genotype_count[genotypes[2]]                 #Count of hetero 2
        hetero = het1 + het2                                #Count of both heteros together
        genotype_count[genotypes[1]] = hetero               
        del genotype_count[genotypes[2]]
        genotypes.remove(genotypes[2])
    
    for key in genotype_count:                          #This is for normalizing the count?
        genotype_count[key] = genotype_count[key]/n_samples
    
    print('Observation', genotype_count)
    genotype_count = collections.OrderedDict(sorted(genotype_count.items()))

    if len(genotypes) == 3:
        allele_A = genotype_count[genotypes[0]] + (genotype_count[genotypes[1]] * 0.5)
        allele_B = genotype_count[genotypes[2]] + (genotype_count[genotypes[1]] * 0.5)
        print('allele:', allele_A)
        print('allele', allele_B)
        if allele_B > allele_A:
            encode = np.array([3,2,1])
        else:
            encode = np.array([1,2,3])
    
    elif len(genotypes) == 2:
        encode = np.array([1,2])
    else:
        encode = np.array([1,2])
    
    genotype_dict = dict(zip(genotypes, encode))
    data[snp] = data[snp].map(genotype_dict)
    print(data[snp].head())
    counter = counter + 1
    if counter>5000:
        break

outfile = 'encoded_snp.csv'
data.to_csv(outfile, sep = ' ')

Streaming output truncated to the last 5000 lines.
Observation {'CT': 0.4921875, 'CC': 0.3828125, 'TT': 0.125}
allele: 0.62890625
allele 0.37109375
usersid
user123     2.0
user285     1.0
user2849    1.0
user1       3.0
user26      1.0
Name: rs7032559, dtype: float64
SNP NUMBER: 4545
Observation {'CT': 0.4453125, 'TT': 0.3828125, 'CC': 0.171875}
allele: 0.39453125
allele 0.60546875
usersid
user123     2
user285     1
user2849    3
user1       3
user26      3
Name: rs703256, dtype: int64
SNP NUMBER: 4546
Observation {'TT': 0.75, 'CT': 0.21875, 'CC': 0.03125}
allele: 0.140625
allele 0.859375
usersid
user123     1
user285     2
user2849    1
user1       1
user26      1
Name: rs7032583, dtype: int64
SNP NUMBER: 4547
Observation {'TT': 0.546875, 'CT': 0.4296875, 'CC': 0.0234375}
allele: 0.23828125
allele 0.76171875
usersid
user123     1
user285     1
user2849    2
user1       1
user26      1
Name: rs7032620, dtype: int64
SNP NUMBER: 4548
Observation {'TT': 0.4609375, 'CT': 0.4140625, 'CC': 

In [ ]:
outfile = 'encoded_snp2.csv'
data.to_csv(outfile, sep = ',')

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import numpy as np
import pandas as pd
import itertools


from datetime import datetime
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [ ]:
from numpy import random

In [ ]:
def convertLabel(data):
	label_maker = LabelEncoder()
	data['Encode'] = label_maker.fit_transform(data['Class'])

def knn(X_train, y_train, X_test):
	scaler = StandardScaler()  
	scaler.fit(X_train)

	X_train = scaler.transform(X_train)  
	X_test = scaler.transform(X_test)

	classifier = KNeighborsClassifier(n_neighbors=k)  
	classifier.fit(X_train, y_train) 
	return classifier.predict(X_test)

In [ ]:
start = datetime.now()
data_file = '/content/drive/MyDrive/Identifying Diseases Using ML /encoded_5000snp2.csv'
data = pd.read_csv(data_file, sep = ',')
data = data.drop('usersid', axis=1)
data = data.replace(np.NaN, 0)


correct = 0
mismatch = 0
y_pred = []
n_features = 400
k = 7
counter = 0
n_samples = 128
y_true = []


snp = data.columns.values
snp = snp[:-1]
X = data[snp].values

In [ ]:
data["Class"] = 0
for i in range(len(data)):
    data["Class"][i] = random.randint(2)

print(data["Class"].nunique())

In [ ]:
convertLabel(data)
y = data['Encode'].values

In [ ]:
d = data.items()
datadict = {}
counter = 1
for item in d:
    datadict[item[0]] = item[1]
    counter = counter+1
    if counter>5000:
        break

In [ ]:
data2 = pd.DataFrame.from_dict(datadict)

In [ ]:
data2["Class"] = data["Class"]

In [ ]:
convertLabel(data2)
y = data2['Encode'].values

In [ ]:
snp = data2.columns.values
snp = snp[:-1]
X = data2[snp].values

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)


for train_index, test_index in loo.split(X):
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]

	fs = RandomForestClassifier(max_features = 500, n_estimators=200, random_state=42)
	fs.fit(X_train, y_train)
	ranking = dict(zip(snp, fs.feature_importances_))
	ranking = dict(itertools.islice(ranking.items(), n_features))
	top_snp = list(ranking.keys())
	top_snp = np.asarray(top_snp)

	X_train = data.drop(test_index)
	X_train = X_train[top_snp].values
	X_test = data.reindex(test_index)
	X_test = X_test[top_snp].values

	tmp = knn(X_train, y_train, X_test)
	
	y_pred.append(tmp[0])
	y_true.append(y_test)
	print (counter, '===========================')
	print('Prediction:', tmp)
	print('True label', y_test[0])

	counter = counter + 1
	

y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print('Number of SNPs:', n_features)
print('Accuracy using of rf-knn', accuracy)
print('Precision using of rf-knn', accuracy)
print('Recall using of rf-knn', accuracy)
print('\nConfusion matrix of k-nearest neighbors (kNN) optimized on the test data:')
print(pd.DataFrame(confusion_matrix(y_true, y_pred),
      columns=['pred_pos', 'pred_neg'], index=['pos', 'neg']))
print('Run-time', datetime.now() - start)

5001 ===========================
Prediction: [1]
True label 0
5002 ===========================
Prediction: [1]
True label 1
5003 ===========================
Prediction: [0]
True label 1
5004 ===========================
Prediction: [0]
True label 1
5005 ===========================
Prediction: [1]
True label 0
5006 ===========================
Prediction: [1]
True label 1
5007 ===========================
Prediction: [1]
True label 0
5008 ===========================
Prediction: [1]
True label 1
5009 ===========================
Prediction: [1]
True label 0
5010 ===========================
Prediction: [0]
True label 1
5011 ===========================
Prediction: [1]
True label 0
5012 ===========================
Prediction: [1]
True label 0
5013 ===========================
Prediction: [1]
True label 1
5014 ===========================
Prediction: [1]
True label 1
5015 ===========================
Prediction: [1]
True label 1
5016 ===========================
Prediction: [0]
True label 1
5017 ===